Declare a numba accelerated function that computes the Halton QRNG
1. The parameter $n$ is an integer of any size
2. The parameter $p$ is a prime number

In [ ]:
"""mc_sphere.ipynb"""

# Cell 01

import numpy as np
from numba import float64, int64, vectorize


@vectorize([float64(int64, int64)], nopython=True)
def halton(n, p):
    h, f = 0, 1
    while n > 0:
        f = f / p
        h += (n % p) * f
        n = int(n / p)
    return h


print(halton(10_000, 2))

Set the total number of dots (samples) to take

In [ ]:
# Cell 02

total_dots = 125_000

print(f"{total_dots = :,}")

Create `total_dots` samples of 3D Cartesian points $(x,y,z)$ using the Halton sequence
1. The Halton QRNG returns a random float [0,1)
2. Subtract that float from 1 which makes the interval flip to become (0,1]\
   This ensures that any points exactly on the *surface* will now contribute to the volume
3. Scale the result so it now falls in the interval [-1, 1]


In [ ]:
# Cell 03

import pandas as pd

x = (1 - halton(np.arange(total_dots), 2)) * 2 - 1
y = (1 - halton(np.arange(total_dots), 3)) * 2 - 1
z = (1 - halton(np.arange(total_dots), 5)) * 2 - 1

pd.DataFrame({"x": x[:5], "y": y[:5], "z": z[:5]})

**Create an array $d$ that contains the distance from the origin $(0,0)$ for every point $(x,y,z)$**\
Leverage the fact the exponentiation and addition operators are "vector aware"

In [ ]:
# Cell 04

d = np.sqrt(x**2 + y**2 + z**2)

pd.DataFrame({"x": x[:5], "y": y[:5], "z": z[:5], "d": d[:5]})

Create arrays of $(x,y)$ coordinates that are "on or inside" vs. "outside" the sphere\
using the Pythagorean distance $d$\
Leverage the ability to `filter` numpy arrays using a conditional expression

In [ ]:
# Cell 05

# On the surface (or inside) the sphere
x_in = x[d <= 1.0]
y_in = y[d <= 1.0]
z_in = z[d <= 1.0]

# Outside the sphere
x_out = x[d > 1.0]
y_out = y[d > 1.0]
z_out = z[d > 1.0]

pd.DataFrame(
    {
        "x_in": x_in[:5],
        "y_in": y_in[:5],
        "z_in": z_in[:5],
        "x_out": x_out[:5],
        "y_out": y_out[:5],
        "z_out": z_out[:5],
    }
)

Display the scatter plot of the Monte Carlo estimation

In [ ]:
# Cell 06

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
ax = plt.axes(projection="3d")
ax.view_init(azim=-72, elev=48)
ax.scatter(x_in, y_in, z_in, color="red", marker=".", s=0.1)
ax.scatter(x_out, y_out, z_out, color="blue", marker=".", s=0.1)
plt.gca().set_aspect("equal")
plt.show()

Calculate the absolute percent error in the area estimation
1. The actual/expected volume of a unit sphere is exactly $\frac{4}{3}\pi$
2. The observed/estimated volume using the Monte Carlo formulation $=8\times\dfrac{dots_{\ inside}}{dots_{\ total}}$
3. The sample volume has length, width, and height of two $\to 2\times 2\times 2=8$


In [ ]:
# Cell 07

act = 4 / 3 * np.pi
est = 8 * np.count_nonzero(d <= 1.0) / total_dots
err = np.abs((est - act) / act)

print(f"dots = {total_dots:,}")
print(f"act = {act:.6f}")
print(f"est = {est:.6f}")
print(f"err = {err:.5%}")